In [1]:
import os 
from dotenv import load_dotenv

#load variables from the .env file
load_dotenv()

# Get the key from the environment
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Gemini API key set successfully.")
else: 
    print("GOOGLE_API_KEY not found in environment variables.")

Gemini API key set successfully.


In [ ]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# Section 2: Model Context Protocol

Model Context Protocol (MCP) is an open standard that lets agents use community-built integrations. Instead of writing your own integrations and API clients, just connect to an existing MCP server.

✅ Access live, external data from databases, APIs, and services without custom integration code

✅ Leverage community-built tools with standardized interfaces

✅ Scale capabilities by connecting to multiple specialized servers

In [9]:
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    ),
    errlog=None, #  <--- This line here
)

print("✅ MCP Tool created")

✅ MCP Tool created


Behind the scenes:

Server Launch: ADK runs npx -y @modelcontextprotocol/server-everything

Handshake: Establishes stdio communication channel

Tool Discovery: Server tells ADK: "I provide getTinyImage" functionality

Integration: Tools appear in agent's tool list automatically

Execution: When agent calls getTinyImage(), ADK forwards to MCP server

Response: Server result is returned to agent seamlessly

Why This Matters: You get instant access to tools without writing integration code!

In [10]:
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="image_agent",
    instruction="USe the MCP Tool to generate images for user queries",
    tools=[mcp_image_server],
)

In [11]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=image_agent)

In [12]:
response = await runner.run_debug("Provide a sample tiny image", verbose=True)


 ### Created new session: debug_session_id

User > Provide a sample tiny image


c:\Python313\Lib\site-packages\google\adk\tools\mcp_tool\mcp_tool.py:88: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


image_agent > [Calling tool: getTinyImage({})]
image_agent > [Tool result: {'content': [{'type': 'text', 'text': 'This is a tiny image:'}, {'type': 'image', 'data': 'iVBORw0KG...]
image_agent > This is a tiny image:



In [13]:
from IPython.display import display, Image as IPImage
import base64

for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, "function_response") and part.function_response:
                for item in part.function_response.response.get("content", []):
                    if item.get("type") == "image":
                        display(IPImage(data=base64.b64decode(item["data"])))

Kaggle MCP Server - For dataset and notebook operations

Kaggle provides an MCP server that lets your agents interact with Kaggle datasets, notebooks, and competition

In [14]:
McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=['y',
                  'mcp-remote',
                  'https://www.kaggle.com/mcp'
            ],
        ),
        timeout=30,
    )
)

What it provides:

📊 Search and download Kaggle datasets

📓 Access notebook metadata

🏆 Query competition information etc.,

GitHub MCP Server - For PR/Issue analysis

In [16]:
from google.adk.tools.mcp_tool.mcp_session_manager import (
    StdioConnectionParams, 
    StreamableHTTPServerParams  # Add this line
)

In [17]:
McpToolset(
    connection_params=StreamableHTTPServerParams(
        url="https://api.githubcopilot.com/mcp/",
        headers={
            "Authorization": f"Bearer {GITHUB_TOKEN}",
            "X-MCP-Toolsets": "all",
            "X-MCP-Readonly": "true"
        },
    ),
)

NameError: name 'GITHUB_TOKEN' is not defined